In [1]:
using Revise
using NonEqDigits

In [2]:
using LaTeXStrings
using GLMakie
using CairoMakie
using JLD2
using Geppetto

In [3]:
includet("../scripts/ma_graphviz.jl")

In [4]:
includet("../scripts/ma_colors.jl")
single_col_width = 324 # corresponds to 3+3/8 in as state in PR guidelines
double_col_width = single_col_width*2
golden_ratio = 1.618
nothing

# All rules with just one ssac

# Bits and bobs

In [3]:
GC.gc()
empty!(Out)
GC.gc()